# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
0.  artist 
1. firstName of user
2. gender of user
3. item number in session
4. last name of user
5. length of the song
6. level (paid or free song)
7. location of the user
8. sessionId
9. song title
10. userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [100]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [101]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [102]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Solutions

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [103]:
q1 = "SELECT artist, song, length FROM music_library WHERE session_id = 338 AND item_in_session = 4"

Our data model will include all requested columns in the selected fielsd, will have the Primary Key the session_id column and will also be clustered by item_in_session column

In [104]:
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [105]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # for some of the columns we need to typecast to int or float
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Select requested data

In [106]:

try:
    rows = session.execute(q1)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [107]:
q2 = "SELECT item_in_session, artist, song, user FROM music_users_library WHERE user_id = 10 AND session_id = 182"

Our data model will include all the selected fields, will have user_id as the Primary Key and will be clustered by session_id and item_in_session columns

In [108]:
query = "CREATE TABLE IF NOT EXISTS music_users_library "
query = query + "( \
    user_id int, \
    session_id int, \
    artist text, \
    song text, \
    user text, \
    item_in_session int, \
    PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [109]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_users_library (user_id, session_id, artist, song, user, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        # for some of the columns we need to typecast to int or float
        # the user first and last name will be composed into one wield
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1] + ' ' + line[4], int(line[3])))

#### Select requested data
Will also show the item_in_session field in order to the the proper song order

In [110]:
try:
    rows = session.execute(q2)
except Exception as e:
    print(e)

for row in rows:
    print (row.item_in_session, row.artist, row.song, row.user)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [111]:
q3 = "SELECT user FROM music_history WHERE song = 'All Hands Against His Own'"

Our data model will include all selected fields, will have the song as the Primary Key, and in order to have uniqueness per user we will cluster by user.
This format will assure that users will not appear twice if they listen multiple times a song.

In [112]:
query = "CREATE TABLE IF NOT EXISTS music_history "
query = query + "( \
    song text, \
    user text, \
    PRIMARY KEY (song, user))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [113]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (song, user)"
        query = query + " VALUES (%s, %s)"
        # the user first and last name will be composed into one wield
        session.execute(query, (line[9], line[1] + ' ' + line[4]))

#### Select requested data

In [114]:
try:
    rows = session.execute(q3)
except Exception as e:
    print(e)

for row in rows:
    print (row.user)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [115]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [116]:
query = "drop table music_users_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [117]:
query = "drop table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [118]:
session.shutdown()
cluster.shutdown()